In [88]:
import pandas as pd
import numpy as np
import re


# def process_code(code):
#     try:
#         return str(code)[:8]
#     except Exception:
#         return None


def sord(df):
    new_df = pd.DataFrame(columns=['time', 'code', 'IM_PQ', 'IM_Q', 'EX_PQ', 'EX_Q'])

    pd.DataFrame(columns=['time', 'code', 'IM_PQ', 'IM_Q', 'EX_PQ', 'EX_Q'])
    
    # 处理进口金额
    import_amount = df[df['指标'] == '进口金额（美元）']
    import_amount = import_amount[['时间', '商品', '数值']].rename(columns={'时间': 'time', '商品': 'code', '数值': 'IM_PQ'})
    
    # 处理进口量（仅第一数量）
    import_quantity = df[df['指标'] == '进口数量（第一数量）']
    import_quantity = import_quantity[['时间', '商品', '数值']].rename(columns={'时间': 'time', '商品': 'code', '数值': 'IM_Q'})
    
    # 处理出口金额
    export_amount = df[df['指标'] == '出口金额（美元）']
    export_amount = export_amount[['时间', '商品', '数值']].rename(columns={'时间': 'time', '商品': 'code', '数值': 'EX_PQ'})
    
    # 处理出口量（仅第一数量）
    export_quantity = df[df['指标'] == '出口数量（第一数量）']
    export_quantity = export_quantity[['时间', '商品', '数值']].rename(columns={'时间': 'time', '商品': 'code', '数值': 'EX_Q'})
    
    # 合并数据
    new_df = (
        import_amount
        .merge(export_amount, on=['time', 'code'], how='outer')
        .merge(import_quantity, on=['time', 'code'], how='outer')
        .merge(export_quantity, on=['time', 'code'], how='outer')
    )
    
    # 填充缺失值为 "空"
    new_df = new_df.fillna(pd.NA)
    new_df.head()
    return new_df


In [89]:
df = pd.read_csv('./09xxxx.cn.csv', encoding='gbk')[:-2]
sorted_df = sord(df)
iv = pd.read_csv('./iv.list.csv')
sorted_df["code"] = sorted_df["code"].str.strip().apply(lambda x: str(x.split()[0]))


In [90]:
# 防止除以零或 NaN 的情况，计算 IM_P
sorted_df["IM_P"] = sorted_df.apply(
    lambda row: row["IM_PQ"] / row["IM_Q"] if row["IM_Q"] not in [0, np.nan] else np.nan, axis=1
)

# 防止除以零或 NaN 的情况，计算 EX_P
sorted_df["EX_P"] = sorted_df.apply(
    lambda row: row["EX_PQ"] / row["IM_Q"] if row["IM_Q"] not in [0, np.nan] else np.nan, axis=1
)

# 从 time 列提取年份并创建 year 列
sorted_df["year"] = sorted_df["time"].str[-4:].astype(int)
# sorted_df = remove2024(sorted_df)

In [91]:
sorted_df['year'] = sorted_df['year'].astype(int)
iv['year'] = iv['year'].astype(int)
df = sorted_df.merge(iv, on='year', how='left')

In [92]:
df["lgdp"] = df["gdp"].apply(np.log)
df["lpop"] = df["pop"].apply(np.log)
df["lfdi"] = df["fdi"].apply(np.log)
df["lexp"] = df['EX_P'].apply(np.log)
df["limp"] = df['IM_P'].apply(np.log)
df["lexq"] = df['EX_Q'].apply(np.log)
df["limq"] = df['IM_Q'].apply(np.log)

In [93]:
df['len'] = df['code'].apply(len)
df['dropped'] = (
        (df["IM_PQ"].isna()) & 
        (df["EX_PQ"].isna()) & 
        (df["IM_Q"].isna()) & 
        (df["EX_Q"].isna())
).astype(int)

df['missing1'] = (
        (df["IM_PQ"].isna()) |
        (df["EX_PQ"].isna()) |
        (df["IM_Q"].isna()) | 
        (df["EX_Q"].isna())
).astype(int)


In [94]:
rate = pd.read_csv('./exc_m.csv', encoding='utf-8')
df = df.merge(rate, on='time', how='left')

In [95]:
df.isin([-np.inf, np.inf]).sum()

time         0
code         0
IM_PQ        0
EX_PQ        0
IM_Q         0
EX_Q         0
IM_P         0
EX_P         0
year         0
fdi          0
gdp          0
pop          0
tea          0
btea         0
gtea         0
lgdp         0
lpop         0
lfdi         0
lexp         0
limp         0
lexq         0
limq        58
len          0
dropped      0
missing1     0
rate         0
dtype: int64

In [96]:
df.replace([np.inf, -np.inf], np.nan, inplace=True)

In [97]:
df['is_tea'] = df['code'].str[:4].isin(['0901', '0902', '0903']).astype(int)
df['i0901'] = df['code'].str[:4].isin(['0901']).astype(int)
df['i0902'] = df['code'].str[:4].isin(['0902']).astype(int)
df['i0903'] = df['code'].str[:4].isin(['0903']).astype(int)

In [98]:
tea_e = pd.read_csv("./tea_sq.csv", encoding='utf-8')
df = df.merge(tea_e, on='year', how='left')

In [99]:
df.to_csv("./09.cn.csv", encoding='utf-8', index=False)


In [100]:
df

,time,code,IM_PQ,EX_PQ,IM_Q,EX_Q,IM_P,EX_P,year,fdi,...,len,dropped,missing1,rate,is_tea,i0901,i0902,i0903,tea_sq,tea_pr
0,01-2017,09011100,NaN,NaN,NaN,NaN,NaN,NaN,2017,13632000.0,...,8,1,1,6.858514,1,1,0,0,2848.72000,246.04000
1,01-2017,09012100,6642370.0,NaN,1897820.0,NaN,3.500000,NaN,2017,13632000.0,...,8,0,1,6.858514,1,1,0,0,2848.72000,246.04000
2,01-2017,09012200,NaN,NaN,NaN,NaN,NaN,NaN,2017,13632000.0,...,8,1,1,6.858514,1,1,0,0,2848.72000,246.04000
3,01-2017,09019010,NaN,NaN,NaN,NaN,NaN,NaN,2017,13632000.0,...,8,1,1,6.858514,1,1,0,0,2848.72000,246.04000
4,01-2017,09019020,NaN,NaN,NaN,NaN,NaN,NaN,2017,13632000.0,...,8,1,1,6.858514,1,1,0,0,2848.72000,246.04000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5854,12-2023,0904,2695.0,NaN,99.0,NaN,27.222222,NaN,2023,16325000.0,...,4,0,1,7.150171,0,0,0,0,3429.95156,354.11234
5855,12-2023,0906,NaN,NaN,NaN,NaN,NaN,NaN,2023,16325000.0,...,4,1,1,7.150171,0,0,0,0,3429.95156,354.11234
5856,12-2023,0909,NaN,NaN,NaN,NaN,NaN,NaN,2023,16325000.0,...,4,1,1,7.150171,0,0,0,0,3429.95156,354.11234
5857,12-2023,0910,NaN,NaN,NaN,NaN,NaN,NaN,2023,16325000.0,...,4,1,1,7.150171,0,0,0,0,3429.95156,354.11234


In [101]:
df["time"].describe()

count        5859
unique         93
top       01-2017
freq           63
Name: time, dtype: object

In [102]:
codes = df['code'].unique()

results = []

start_year = 2017
end_year = 2023

In [103]:
for c in codes:
    # 首先判断该code是否长度为6，不是则跳过
    if len(c) != 6:
        continue
    # span_length 为1到7年
    for span_length in range(1, end_year - start_year + 2):
        # 根据span_length确定开始年范围
        # 例如span_length=1时，从2017到2023每年都可作为start_yr
        # span_length=7时，仅2017可作为start_yr
        for start_yr in range(start_year, end_year - span_length + 2):
            end_yr = start_yr + span_length - 1
            
            # 筛选对应code、对应年份区间的数据子集
            subset = df[
                (df['code'] == c) &
                (df['year'] >= start_yr) &
                (df['year'] <= end_yr) &
                (df['len'] == 6)
            ][['limp','limq','tea']].dropna()
            
            # 若数据量不足或全部空值，则无法计算协方差
            if len(subset) < 2:
                elasticity = np.nan
            else:
                # 利用pandas的cov()计算协方差矩阵
                # subset.cov() 将返回limp, limq, tea三者的协方差矩阵(DataFrame)
                cov_matrix = subset[['limp','limq','tea']].cov()
                
                cov_limp_tea = cov_matrix.loc['limp', 'tea'] if ('limp' in cov_matrix.index and 'tea' in cov_matrix.columns) else np.nan
                cov_limq_tea = cov_matrix.loc['limq', 'tea'] if ('limq' in cov_matrix.index and 'tea' in cov_matrix.columns) else np.nan
                
                # 检查分母是否为0或NaN
                if pd.isna(cov_limp_tea) or pd.isna(cov_limq_tea) or cov_limq_tea == 0:
                    elasticity = np.nan
                else:
                    elasticity = cov_limp_tea / cov_limq_tea
            
            results.append({
                'code': c,
                'start_year': start_yr,
                'end_year': end_yr,
                'span_years': span_length,
                'elasticity': elasticity
            })

In [104]:
# 将结果转换为DataFrame
edf = pd.DataFrame(results, columns=['code','start_year','end_year','span_years','elasticity'])

# edf即为最终结果DataFrame
print(edf.head())

     code  start_year  end_year  span_years  elasticity
0  090111        2017      2017           1         NaN
1  090111        2018      2018           1         NaN
2  090111        2019      2019           1         NaN
3  090111        2020      2020           1         NaN
4  090111        2021      2021           1         NaN


In [105]:
conditions = [
    (edf['end_year'] <= 2019), 
    (edf['start_year'] >= 2020),
    (edf['start_year'] <= 2019) & (edf['end_year'] >= 2020)
]
choices = [0, 1, -1]

edf['covid'] = np.select(conditions, choices, default=np.nan)

In [106]:
edf['code'] = edf['code'].astype(str)
edf.to_csv("./09_edf.csv", encoding='utf-8', index=False)

In [107]:
event_df = edf[((edf['start_year'] == 2020) | (edf['end_year'] == 2019))].copy()

# 按照要求的列重新命名或选择
# code列为str
event_df['code'] = event_df['code'].astype(str)

# 创建最终需要的列：code, span, covid, ela
event_df = event_df[['code', 'span_years', 'covid', 'elasticity']]

# 重命名列
event_df = event_df.rename(columns={
    'span_years': 'span',
    'elasticity': 'ela'
})

# 将结果导出为CSV文件
event_df.to_csv('event.csv', index=False)

In [108]:
event_df

,code,span,covid,ela
2,090111,1,0.0,NaN
3,090111,1,1.0,NaN
8,090111,2,0.0,NaN
10,090111,2,1.0,0.250891
13,090111,3,0.0,-0.137366
...,...,...,...,...
652,091099,2,0.0,0.152607
654,091099,2,1.0,0.285796
657,091099,3,0.0,-0.830365
660,091099,3,1.0,0.395006


In [109]:
# 假设当前有 event_df:
# 列包括: code(str), span(int), covid(int), ela(float)

# 首先确保 code 和 span 是合适的键
event_df['code'] = event_df['code'].astype(str)

# 使用 pivot 将 covid 的值(0或1) 变成列
wide_df = event_df.pivot_table(
    index=['code', 'span'], 
    columns='covid', 
    values='ela'
).reset_index()

# 此时 wide_df 的列结构大致为:
# code, span, 0, 1
# 其中0列对应疫情前( before trade )的ela，1列对应疫情后 ( after trade )的ela

# 重命名列
wide_df = wide_df.rename(columns={0: 'btd', 1: 'atd'})

# 如果出现某些 code-span 组合只有 before 或 after 数据，需要斟酌如何处理缺失值。
# 假设原数据总是成对存在，否则btd或atd可能为NaN。

# 最终的数据结构:
# code | span | btd | atd

print(wide_df.head())

# 可以将最终结果输出为 CSV 文件
wide_df.to_csv('./wide.csv', encoding='utf-8', index=False)

covid    code  span       btd       atd
0      090111     2       NaN  0.250891
1      090111     3 -0.137366  0.250891
2      090111     4       NaN  0.250891
3      090121     2 -0.361059 -0.123108
4      090121     3 -0.057002 -0.102985


In [110]:
wide_df

covid,code,span,btd,atd
0,090111,2,NaN,0.250891
1,090111,3,-0.137366,0.250891
2,090111,4,NaN,0.250891
3,090121,2,-0.361059,-0.123108
4,090121,3,-0.057002,-0.102985
5,090121,4,NaN,-0.239942
6,090122,2,0.092321,-1.385385
7,090122,3,0.092321,2.931460
8,090122,4,NaN,2.931460
9,090210,1,0.000000,0.725000
